In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
import cv2 as cv;
from google.colab.patches import cv2_imshow
from tensorflow.keras import layers, models
from google.colab import drive
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Rescaling
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
glioma_tumor_path = "/content/drive/MyDrive/Data/Training/glioma_tumor"
meninigioma_tumor = "/content/drive/MyDrive/Data/Training/meningioma_tumor"
no_tumor_path = "/content/drive/MyDrive/Data/Training/no_tumor"
pituitary_tumor_path = "/content/drive/MyDrive/Data/Training/pituitary_tumor"
lioma_tumor_path_ts = "/content/drive/MyDrive/Data/Testing/glioma_tumor"
meninigioma_tumor_ts = "/content/drive/MyDrive/Data/Testing/meningioma_tumor"
no_tumor_path_ts = "/content/drive/MyDrive/Data/Testing/no_tumor"
pituitary_tumor_path_ts = "/content/drive/MyDrive/Data/Testing/pituitary_tumor"

In [3]:
train_folder = [
      glioma_tumor_path,
      lioma_tumor_path_ts,
      meninigioma_tumor,
      meninigioma_tumor_ts,
      pituitary_tumor_path,
      pituitary_tumor_path_ts,
      no_tumor_path,
      no_tumor_path_ts,
]
y = [0,1,2,3,0,1,2,3]

X = []
Y = []

currIndex = 0;
for folder_path in train_folder:
  files = os.listdir(folder_path)

  for file in files:
      if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
          image_cv2 = cv.resize(cv.imread(os.path.join(folder_path, file),0),(150,150))
          X.append(image_cv2)
          Y.append(y[currIndex])
          X.append(cv.flip(image_cv2,-1))
          Y.append(y[currIndex])
          X.append(cv.flip(image_cv2,1))
          Y.append(y[currIndex])
          X.append(255 - image_cv2)
          Y.append(y[currIndex])
  currIndex += 1

In [4]:
X = np.array(X).astype(np.int64)
y = np.array(Y).astype(np.int64)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.1,random_state=5)

In [6]:
model = models.Sequential()


model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='relu'))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(4, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_test, y_test),callbacks=[early_stopping])

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


368/368 [==============================] - 22s 35ms/step - loss: 1.4376 - accuracy: 0.5086 - val_loss: 1.3987 - val_accuracy: 0.5536
Epoch 2/30
368/368 [==============================] - 11s 29ms/step - loss: 1.3898 - accuracy: 0.5233 - val_loss: 1.3616 - val_accuracy: 0.5643
Epoch 3/30
111/368 [========>.....................] - ETA: 7s - loss: 1.3764 - accuracy: 0.5203

KeyboardInterrupt: ignored

In [ ]:
prd = model.predict(X_test)

In [ ]:
prd

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.evaluate(X_train,y_train)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()